In [17]:
#--Librerias--#
import pandas as pd 
import numpy as np
from sklearn.model_selection import RandomizedSearchCV # búsqueda aleatoria de hiperparámetros 
import joblib  ### para guardar y cargar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier # Arboles de decision 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler ## escalar variable
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier #Modelo de clasificacion
from sklearn.metrics import confusion_matrix #### Matriz de confusion 
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns #####Graficos
from sklearn.metrics import accuracy_score #### medir precision del modelo 
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RidgeClassifier
import funciones


In [2]:
###----Lectura de datos----###
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/Predicci-n-precio-de-m-viles-/refs/heads/main/Datos/df_fin", sep=",")

# Dummies

In [3]:
# Conversion; categoricas a dummies
df = pd.get_dummies(df, dtype = int)

In [4]:
# Separamos variables explicativas - variable objetivo
X = df.drop('price_range',axis=1)
y = df['price_range']

print(X.shape)
print(y.shape)

(1958, 20)
(1958,)


# Selección de variables

In [5]:
### Modelos
m_rl = LogisticRegression() 
m_rf = RandomForestClassifier()
m_gbc = GradientBoostingClassifier()

*1. Selec From Model*


*1.1. Ridge Classifier*

In [6]:
# Selector de variables con Ridge
sel_ = SelectFromModel(RidgeClassifier(alpha = 0.1, random_state = 24), max_features = 20)
sel_.fit(X, y)
print(sel_.estimator_.coef_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_ridge = X.iloc[:,X_new]
df_ridge.info()

[[-3.37135427e-04 -7.94395112e-03 -7.59286402e-03 -1.86900918e-03
   1.65311456e-02 -1.75401406e-04 -9.62612774e-02  9.17783901e-03
  -2.73335634e-04 -4.79027085e-03  1.50232208e-02 -7.93439099e-03
   8.78656103e-03  3.34850320e-02  2.05425412e-02  7.34641963e-01
  -3.81707370e-01 -3.52934593e-01  8.42247033e-03 -4.30226468e-03]
 [-3.29565844e-05  8.89185631e-03  3.35610390e-02  8.68136639e-04
   2.17344703e-02  5.01204523e-04  1.20154782e-01 -2.10222076e-02
  -8.52644735e-06  5.44716889e-03  1.91767822e-02  5.06810827e-03
  -4.24599548e-02 -7.51967363e-02 -5.69430634e-02  3.50614851e-01
   7.76342675e-02 -4.28249119e-01  2.14170197e-01  2.08180299e-01]
 [ 3.18866778e-05 -7.90745498e-03 -1.36934749e-02  4.02178344e-03
  -6.35453246e-02 -1.52939222e-03 -7.33368229e-03  1.01815778e-02
  -1.48961716e-05 -1.92119234e-03 -6.52614624e-02 -1.62948560e-02
  -1.21418086e-02  3.42817027e-03  5.85279146e-02 -4.50757411e-01
   6.81564273e-01 -2.30806862e-01 -2.33633969e-01 -1.31297015e-01]
 [ 3.38

*1.2. Gradient Boosting*

In [7]:
# Selector de variables con Gradient Boosting
sel_ = SelectFromModel(GradientBoostingClassifier(n_estimators = 300, random_state=42))
sel_.fit(X, y)
print(sel_.estimator_.feature_importances_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_gb = X.iloc[:,X_new]
df_gb.head()

[0.17455475 0.00117526 0.00141278 0.01515969 0.00336483 0.03035032
 0.0100731  0.00922583 0.13494548 0.02227413 0.00290676 0.00096382
 0.00089498 0.00338788 0.00254654 0.1805043  0.12019519 0.28002119
 0.00242055 0.00362263]


,battery_power,px_width,ram_cat_2GB,ram_cat_3GB,ram_cat_4GB
0,842,756,0,1,0
1,1021,1988,0,1,0
2,563,1716,0,1,0
3,615,1786,0,1,0
4,1821,1212,1,0,0


In [19]:
 from sklearn.metrics import confusion_matrix, classification_report
 # Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

    # Entrenar el modelo de regresión logística
lr = LogisticRegression(max_iter=1000, class_weight="balanced", multi_class='multinomial', solver='lbfgs', random_state=42)
lr.fit(X_train, y_train)

    # Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

    # Métricas para el conjunto de entrenamiento
print('-' * 30, 'TRAIN', '-' * 30)
print(classification_report(y_train, y_pred_train))

    # Métricas para el conjunto de prueba
print('-' * 30, 'TEST', '-' * 30)
print(classification_report(y_test, y_pred_test))

resultados = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'y_pred_train': y_pred_train,
        'y_pred_test': y_pred_test,
        'y_pred_prob_train': y_pred_prob_train,
        'y_pred_prob_test': y_pred_prob_test,
    }



------------------------------ TRAIN ------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       391
           1       0.93      0.89      0.91       393
           2       0.79      0.80      0.79       383
           3       0.88      0.90      0.89       399

    accuracy                           0.90      1566
   macro avg       0.90      0.90      0.90      1566
weighted avg       0.90      0.90      0.90      1566

------------------------------ TEST ------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       0.90      0.94      0.92        93
           2       0.79      0.78      0.78       107
           3       0.84      0.83      0.84        92

    accuracy                           0.88       392
   macro avg       0.88      0.88      0.88       392
weighted avg       0.88      0.88      0.88       

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
